In [1]:
import sys
import os
from pathlib import Path

# Get absolute path to project root
project_root = Path(os.getcwd()).resolve()
if project_root.name != "ncg87-blockchain_tracker":
    project_root = project_root.parent.parent  # Adjust if running from a subdirectory

sys.path.append(str(project_root))

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config import Settings

from database import MongoDatabase, MongoInsertOperations, MongoQueryOperations
from database import SQLDatabase, SQLInsertOperations, SQLQueryOperations
from chains import BaseChainProcessor, BaseChainQuerier
import json
import web3



In [2]:

sql_db = SQLDatabase()
querier = BaseChainQuerier()
mongo_db = MongoDatabase()
processor = BaseChainProcessor(sql_db, mongo_db, querier)



In [3]:
query = "SELECT * FROM evm_contract_abis WHERE network = 'Base'"
sql_db.cursor.execute(query)
contract_abis = sql_db.cursor.fetchall()


In [4]:
contract_abis[100]

RealDictRow([('network', 'Base'),
             ('contract_address',
              '0x7860805613b070E90c45bE5a33330142a9D2850B'),
             ('abi',
              '[{"inputs": [{"internalType": "string", "name": "_name", "type": "string"}, {"internalType": "string", "name": "_symbol", "type": "string"}, {"internalType": "uint8", "name": "_decimals", "type": "uint8"}, {"internalType": "uint256", "name": "_maxSupply", "type": "uint256"}], "stateMutability": "nonpayable", "type": "constructor"}, {"anonymous": false, "inputs": [{"indexed": true, "internalType": "address", "name": "owner", "type": "address"}, {"indexed": true, "internalType": "address", "name": "spender", "type": "address"}, {"indexed": false, "internalType": "uint256", "name": "value", "type": "uint256"}], "name": "Approval", "type": "event"}, {"anonymous": false, "inputs": [{"indexed": true, "internalType": "address", "name": "previousOwner", "type": "address"}, {"indexed": true, "internalType": "address", "name": "newOw

In [5]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
from functools import partial

async def process_all_contracts():
    # Create a thread pool - adjust max_workers as needed
    thread_pool = ThreadPoolExecutor(max_workers=10)
    loop = asyncio.get_event_loop()
    
    # Create tasks that run in thread pool
    tasks = [
        loop.run_in_executor(
            thread_pool,
            partial(asyncio.run, processor._process_contract(contract['contract_address'], json.loads(contract['abi'])))
        )
        for contract in contract_abis
    ]
    
    # Gather results
    results = await asyncio.gather(*tasks)
    contract_info = [result for result in results if result]
    for result in contract_info:
        print(result)
    
    thread_pool.shutdown()
    return contract_info

# Run the async function

: 

In [6]:
contract_info = await process_all_contracts()

Error processing contract <web3._utils.datatypes.Contract object at 0x00000239BF56F350>: ('The abi for this contract contains no function definitions. ', 'Are you sure you provided the correct contract abi?')
Error processing contract <web3._utils.datatypes.Contract object at 0x00000239BC311010>: ('The abi for this contract contains no function definitions. ', 'Are you sure you provided the correct contract abi?')
Error processing contract <web3._utils.datatypes.Contract object at 0x00000239DE626C90>: ('The abi for this contract contains no function definitions. ', 'Are you sure you provided the correct contract abi?')
Error processing contract <web3._utils.datatypes.Contract object at 0x00000239BED2BDD0>: cannot access local variable 'token1_info' where it is not associated with a value
Error processing contract <web3._utils.datatypes.Contract object at 0x00000239BC311CD0>: cannot access local variable 'token1_info' where it is not associated with a value
Error processing contract <we